In [1]:
import pandas as pd

# 讀取CSV檔案
csv_file_path = 'instagram/20230404-20240404.csv' 
file=open(csv_file_path,"rb")  
# 直接使用文件路径读取CSV文件
df = pd.read_csv(csv_file_path,encoding='latin-1',dtype=str)


print("預計爬取清單\n",df['URL'])

預計爬取清單
 0      https://www.instagram.com/p/C4Gj2HrtgVL/
1      https://www.instagram.com/p/C374ckFuwVv/
2      https://www.instagram.com/p/C1mNtR_Mx57/
3      https://www.instagram.com/p/C2bL7B0OP4U/
4      https://www.instagram.com/p/C2KV-mkMRbG/
                         ...                   
873    https://www.instagram.com/p/C2pYgT6rBR_/
874    https://www.instagram.com/p/C4wsFz9u5qz/
875    https://www.instagram.com/p/C3dsdJ5rABF/
876    https://www.instagram.com/p/C5Rbgvuu-m1/
877    https://www.instagram.com/p/C0zYuHfOILP/
Name: URL, Length: 878, dtype: object


C:\Users\g11350025\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\g11350025\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
import os

# 設定你想要檢查和創建的資料夾路徑
folder_path = csv_file_path.replace('.csv','/')

# 檢查資料夾是否存在
if not os.path.exists(folder_path):
    # 如果資料夾不存在，則創建它
    os.makedirs(folder_path)
    print(f"資料夾 '{folder_path}' 已創建。")
else:
    # 如果資料夾已存在，則輸出提示信息
    print(f"資料夾 '{folder_path}' 已存在。")

資料夾 'instagram/20230404-20240404/0-7999error2/' 已存在。


一次爬太多好像會被鎖 約60張

In [ ]:
#一體程式
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import time

comtext = []
error = []
imgid = 0

#要用id改起始位置要改
start_post_id = 0 
start_index = 0

num2 = 0 #爬這個迴圈到60就停

errordf = {
    'id': [],
    'URL': [],
    'error': []
}

if start_index is not None:
    
    for i in df['URL'][588:]: #從紀錄id繼續往下跑
    #for i in df['URL']: #從頭

        imgid = i.split('p/')[1].split('/')[0]  # 提取图片ID
        imgpath = f"{folder_path}{imgid}.jpg"
        print('圖片id',imgid)


        # 检查图片是否已经存在
        if os.path.exists(imgpath):
            print(f"圖片 {imgid} 已存在，跳過下載。")
            continue
            
        #爬超過60張圖先休4小時，避免被ban
        if num2 >= 60:
            start_post_id = imgid
            print('超過60張，圖片id',start_post_id)
            #break
            time.sleep(7200)
            num2 = 0

        try:
            num2 += 1
            driver = webdriver.Chrome()
            # 請替換成你要訪問的網頁的URL
            driver.get(i)
            #等待網站載入 等6秒
            driver.implicitly_wait(6)


            ##取圖片 
#             t = driver.find_element(By.XPATH,
#                                     '/html/body/div[2]/div/div/div[2]/div/div/div[1]/section/main/div/div[1]/article/div/div[1]/div/div/div')
            t = driver.find_element(By.XPATH,
                                    '/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div[2]/section/main/div/div[1]/article/div/div[2]/div/div/div')
    
            t1 = t.get_attribute("outerHTML")
            #print('t1\n',t1)

            # 使用 BeautifulSoup 解析 HTML 并获取 src 属性
            soup = BeautifulSoup(t1, 'html.parser')
            img_url = soup.img['src']  # 提取图片的 URL
            print('img_url\n',img_url)

            pic = requests.get(img_url)
            print('pic\n',pic)
            img2 = pic.content #圖片裡的內容
            imgpath = csv_file_path.replace('.csv','/')
            imgpath = imgpath + str(imgid) + '.jpg'
            print(imgpath)
            pic_out = open(imgpath,'wb') 
            pic_out.write(img2) #將get圖片存入img2
            pic_out.close() #關閉檔案(很重要)
            ##取圖片


            ##取評論者 評論內容 評論時間 評論按讚數 存成csv
            child_elements_comment = 0
            child_elements_name = 0

            name_com = []

            # 使用CSS选择器定位父元素，返回一个元素列表
            parent_elements = driver.find_elements(By.CSS_SELECTOR, '._a9zr')

            # 遍历所有父元素
            for parent in parent_elements:
                #print(parent.text,'\n------------\n') 下面失敗的話存這個

                # 在每个父元素下查找评论子元素
                child_elements_comment = parent.find_element(By.CLASS_NAME, '_a9zs')
                # 在每个父元素下查找名字子元素
                child_elements_name = parent.find_element(By.CLASS_NAME, '_a9zc') 

                #讚  
                good_element = parent.find_element(By.CLASS_NAME,
                                                   'x9f619.xjbqb8w.x78zum5.x168nmei.x13lgxp2.x5pf9jr.xo71vjh.x1xmf6yo.x12nagc.x1n2onr6.x1plvlek.xryxfnj.x1c4vz4f.x2lah0s.xdt5ytf.xqjyukv.x1qjc9v5.x1oa3qoh.x1nhvcw1')
                good_element = good_element.text.split('週')[1].split('個讚')[0]
                #時間
                time_element = parent.find_element(By.CSS_SELECTOR, 'time._a9ze._a9zf')
                full_date = time_element.get_attribute('title')

                print("Name:", child_elements_name.text)
                print("Comment:", child_elements_comment.text)
                print("time:",full_date)
                print("likes:",good_element)
                print('-------')

                name_com.append({
                    'name': child_elements_name.text,
                    'comment': child_elements_comment.text,
                    'time': full_date,
                    'likes': good_element
                })

            #個別存評論    
            # 将数据转换成DataFrame
            dfok = pd.DataFrame(name_com)
            print(dfok)

            # 将DataFrame保存为CSV文件
            idimgcsv = folder_path + str(imgid) + '.csv' 
            dfok.to_csv(idimgcsv, index=False,encoding='utf-8-sig')
            #個別存評論 

            driver.close()








        except Exception as e:
            errordf['id'].append(imgid)
            errordf['URL'].append(i)
            errordf['error'].append(f"发生错误：{e}")

            #error.append(imgid) 
            print('失敗',imgid,i)
            print(f"发生错误：{e}")
        time.sleep(6)

    from pandas.core.frame import DataFrame
    # 将数据转换成DataFrame
    dfer = pd.DataFrame(errordf)
    print(len(errordf),dfer)

    #存起來 失敗
    nameerror = csv_file_path.replace('.csv','/') + 'error.csv'
    # 将DataFrame保存为CSV文件
    dfer.to_csv(nameerror, index=False,encoding='utf-8-sig')